In [1]:
using SymEngine
include("../dynamics/lagrange.jl")

lagrange

In [2]:
# Deriving the dynamics

@vars l1 l2 lc1 lc2 q1 q2 dq1 dq2 ddq1 ddq2 g m1 m2 I1 I2 t1 t2 ml Il

(l1, l2, lc1, lc2, q1, q2, dq1, dq2, ddq1, ddq2, g, m1, m2, I1, I2, t1, t2, ml, Il)

In [3]:
# Define the keypoints of the arm
q   = [q1;q2]
dq  = [dq1;dq2]
ddq = [ddq1;ddq2]

ihat = [1;0;0]
jhat = [0;1;0]
khat = cross(ihat,jhat)


r1 = 0.5*l1*(cos(q1)*ihat + sin(q2)*jhat)
r2 = 2*r1 + 0.5*l2*(cos(q1 + q2)*ihat + sin(q1 + q2)*jhat)

# Force of load
# rl = 2*r1 + 2*r2
# Fl = -ml*g*jhat

println("Link 1: ", r1)
println("Link 2: ", r2)

Link 1: SymEngine.Basic[0.5*l1*cos(q1), 0.5*l1*sin(q2), 0.0]
Link 2: SymEngine.Basic[1.0*l1*cos(q1) + 0.5*cos(q1 + q2)*l2, 1.0*l1*sin(q2) + 0.5*sin(q1 + q2)*l2, 0]


In [4]:
# Derivative of linkage velocities
dr1 = lagrange.ddt(r1,q,dq,ddq)
dr2 = lagrange.ddt(r2,q,dq,ddq)

println("Link 1: ", dr1)
println("Link 2: ", dr2)

Link 1: SymEngine.Basic[-0.5*l1*sin(q1)*dq1, 0.5*l1*cos(q2)*dq2, 0]
Link 2: SymEngine.Basic[dq1*(-1.0*l1*sin(q1) - 0.5*sin(q1 + q2)*l2) - 0.5*sin(q1 + q2)*l2*dq2, dq2*(1.0*l1*cos(q2) + 0.5*cos(q1 + q2)*l2) + 0.5*cos(q1 + q2)*l2*dq1, 0]


In [5]:
# Generalized Forces
Q1 = lagrange.M2Q(t1*khat,dq1*khat, dq)
Q2 = lagrange.M2Q(t2*khat,dq2*khat, dq)
# Ql = lagrange.F2Q(rl,Fl,q)

# Q = [Q1[i] + Q2[i] + Ql[i] for i in 1:2]
Q = Q1 .+ Q2

println("Generalized Torque: ", Q)

Generalized Torque: SymEngine.Basic[t1, t2]


In [6]:
# Kinetic and Potential Energy

T1 = 0.5*m1*dot(r1,r1) + 0.5*I1*dot(dq1,dq1)
T2 = 0.5*m2*dot(r2,r2) + 0.5*I2*dot(dq2,dq2)
T  = T1 + T2

V1 = m1*g*dot(r1,-jhat)
V2 = m2*g*dot(r2,-jhat)
V  = V1 + V2

# Lagrangian
L  = T - V

0.5*I1*dq1^2 + 0.5*I2*dq2^2 + 0.5*m2*((1.0*l1*sin(q2) + 0.5*sin(q1 + q2)*l2)^2 + (1.0*l1*cos(q1) + 0.5*cos(q1 + q2)*l2)^2) + 0.5*(0.0 + 0.25*l1^2*sin(q2)^2 + 0.25*l1^2*cos(q1)^2)*m1 - (-g*m2*(1.0*l1*sin(q2) + 0.5*sin(q1 + q2)*l2) - 0.5*g*l1*m1*sin(q2))

In [10]:
# Generate Equations of motion
accel = lagrange.ddt(lagrange.jacobian(L,dq)',q,dq,ddq)
state = -lagrange.jacobian(L,q)'

sys =  accel + state - Q


println("Accel: ",accel)
println("\nState: ",state)
println("\nGeneralized: ",Q)
# println("Equations of motion: ", eom)

Accel: SymEngine.Basic[1.0*I1*ddq1, 1.0*I2*ddq2]

State: SymEngine.Basic[-(0.5*m2*(2*(-1.0*l1*sin(q1) - 0.5*sin(q1 + q2)*l2)*(1.0*l1*cos(q1) + 0.5*cos(q1 + q2)*l2) + 1.0*cos(q1 + q2)*l2*(1.0*l1*sin(q2) + 0.5*sin(q1 + q2)*l2)) + 0.5*g*cos(q1 + q2)*l2*m2 - 0.25*l1^2*m1*sin(q1)*cos(q1)); -(0.5*m2*(2*(1.0*l1*sin(q2) + 0.5*sin(q1 + q2)*l2)*(1.0*l1*cos(q2) + 0.5*cos(q1 + q2)*l2) - 1.0*sin(q1 + q2)*l2*(1.0*l1*cos(q1) + 0.5*cos(q1 + q2)*l2)) + g*m2*(1.0*l1*cos(q2) + 0.5*cos(q1 + q2)*l2) + 0.5*g*l1*m1*cos(q2) + 0.25*l1^2*m1*sin(q2)*cos(q2))]

Generalized: SymEngine.Basic[t1, t2]


In [11]:
# Rewrite as ss model
type(A)
type(B)

A = lagrange.jacobian(g,ddq)
b = A*ddq - sys

ddq2 = A\b

z  = [q;dq]
dz = [dq;ddq2]

println("Z: ", z)
println("dZ: ", dz)

LoadError: [91mDimensionMismatch("dimensions must match")[39m

In [10]:
using DifferentialEquations
using Plots

In [12]:
# Setup dynamics

function manipulator_weight(dz, z, p, t)
    sym_z  = p[1]
    sym_dz = p[2]
    
    dz[1] = subs(sym_z[1],dq=>z[1])
    dz[2] = subs(sym_z[2],dq=>z[2])
    dz[3] = subs(sym_dz[1], dq => dz[1], q => dz[2])
    dz[4] = subs(sym_dz[1], dq => dz[1], q => dz[2])
    
end

manipulator_weight (generic function with 1 method)